In [1]:
from src.api import DataAPI , Predictor

DataAPI.update()
DataAPI.prepare_train_data()
Predictor.update_factors()

Use device name: NVIDIA GeForce RTX 4090
Unpack Update Files
['/home/mengkjin/Workspace/SharedFolder/DB_updater.240625000707.tar', '/home/mengkjin/Workspace/SharedFolder/DB_updater.240626010635.tar', '/home/mengkjin/Workspace/SharedFolder/DB_updater.240626233607.tar', '/home/mengkjin/Workspace/SharedFolder/DB_updater.240628075339.tar', '/home/mengkjin/Workspace/SharedFolder/DB_updater.240629000845.tar', '/home/mengkjin/Workspace/SharedFolder/DB_updater.240629090428.tar', '/home/mengkjin/Workspace/SharedFolder/DB_updater.240702075218.tar', '/home/mengkjin/Workspace/SharedFolder/DB_updater.240703003454.tar', '/home/mengkjin/Workspace/SharedFolder/DB_updater.240703004031.tar', '/home/mengkjin/Workspace/SharedFolder/DB_updater.240703004531.tar', '/home/mengkjin/Workspace/SharedFolder/DB_updater.240703232626.tar', '/home/mengkjin/Workspace/SharedFolder/DB_updater.240704091024.tar', '/home/mengkjin/Workspace/SharedFolder/DB_updater.240704091350.tar', '/home/mengkjin/Workspace/SharedFolder/DB

In [1]:
from src.api import Predictor
Predictor.update_factors()

Finish model [gru_day] predicting!
Finish model [gruRTN_day] predicting!
Finish model [gruRES_day] predicting!
--------------------------------------------------------------------------------


In [2]:
from src.api import ModelTestor
ModelTestor.new('ts_mixer').try_metrics()

Load  2 DataBlocks ...

... cost 0.22 secs
Align 2 DataBlocks ...... cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
x shape is torch.Size([5064, 30, 6])
y shape is torch.Size([5064, 1])
Test Forward Success
metrics :  Metrics.MetricOutput(loss=tensor(1.0688, grad_fn=<AddBackward0>), score=-0.015420470386743546, loss_item=1.068833589553833, penalty=0.0, losses=tensor(1.0688, grad_fn=<ExpBackward0>))
Test Metrics Success
